In [1]:
# Import libraries

import os

import cv2

import tensorflow as tf

import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix

import seaborn as sns

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization, Activation, Add, Input, Concatenate, GlobalAveragePooling2D

from tensorflow.keras.models import Model

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.losses import CategoricalCrossentropy

In [2]:
# Import necessary libraries

import os

import cv2

import tensorflow as tf

import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix

import seaborn as sns

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization, Activation, Add, Input, Concatenate, GlobalAveragePooling2D, AveragePooling2D, DepthwiseConv2D, Multiply

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow.keras.callbacks import ReduceLROnPlateau

In [3]:
# Load data

X_train = np.load('/kaggle/input/kalki-solar123/archana/X_train_n12c.npy')

y_train = np.load('/kaggle/input/kalki-solar123/archana/y_train_n12c.npy')

X_val = np.load('/kaggle/input/kalki-solar123/archana/X_val_n12c.npy')

y_val = np.load('/kaggle/input/kalki-solar123/archana/y_val_n12c.npy')

X_test = np.load('/kaggle/input/kalki-solar123/archana/X_test_n12c.npy')

y_test = np.load('/kaggle/input/kalki-solar123/archana/y_test_n12c.npy')

In [4]:
# Check the shape of target labels

print(f"Shape of y_train: {y_train.shape}")



# Squeeze-and-Excitation (SE) Block

def se_block(input_tensor, ratio=8):

    filters = input_tensor.shape[-1]

    se = GlobalAveragePooling2D()(input_tensor)         #ly8

    #se = Dense(filters // ratio, activation='relu')(se) #ly9

    se = Dense(filters, activation='sigmoid')(se)       #ly10

    return Multiply()([input_tensor, se])               #ly11


Shape of y_train: (64000, 12)


In [5]:
# Enhanced Pyramid Block with Depthwise Separable Convolution and SE Block

def enhanced_pyramid_block(x, filters):

    # Depthwise Separable Conv Layer

    x = DepthwiseConv2D((3, 3), padding='same', depth_multiplier=1, activation='relu')(x)  #ly4

    x = Conv2D(filters, (1, 1), activation='relu')(x)                                      #ly5

    x = BatchNormalization()(x)                                                            #ly6

    

    # Second Conv Layer with SE Block

    x = Conv2D(filters, (3, 3), padding='same', activation='relu')(x)                      #ly7

    x = se_block(x)

    x = BatchNormalization()(x)

    

    # Max Pooling Layer

    x = MaxPooling2D(pool_size=(2, 2))(x)

    return x

In [6]:
# Adaptive Attention Pyramid Network (AAPN)

def adaptive_attention_pyramid_net(input_shape=(40, 24, 1), num_classes=12):

    input_tensor = Input(shape=input_shape)

    

    # Initial Conv Layer

    x = Conv2D(32, (3, 3), padding='same', activation='relu')(input_tensor)  #ly2

    x = BatchNormalization()(x)                                              #ly3

    

    # First Adaptive Pyramid Block

    x = enhanced_pyramid_block(x, filters=64)

    

    # Second Adaptive Pyramid Block

    x = enhanced_pyramid_block(x, filters=128)

    

    # Third Adaptive Pyramid Block

    x = enhanced_pyramid_block(x, filters=256)

    

    # Global Average Pooling

    x = GlobalAveragePooling2D()(x)

    

    # Fully connected layer with 128 units

    x = Dense(128, activation='relu')(x)

    

    # Increased Dropout layer for better regularization

    x = Dropout(0.4)(x)

    

    # Output layer (Class probabilities)

    output_tensor = Dense(num_classes, activation='softmax')(x)

    

    # Build and compile the model

    model = Model(inputs=input_tensor, outputs=output_tensor)

    model.compile(optimizer=Adam(learning_rate=0.001),

                  loss=CategoricalCrossentropy(label_smoothing=0.1),

                  metrics=['accuracy'])

    return model

In [7]:
# Define the model

input_shape = (40, 24, 1)

num_classes = 12

model = adaptive_attention_pyramid_net(input_shape=input_shape, num_classes=num_classes)



# Model summary

model.summary()



# Define learning rate scheduler

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 40, 24, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 40, 24,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 40, 24,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d    │ (None, 40, 24,    │        320 │ batch_normalizat… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 40, 24,    │      2,112 │ depthwise_conv2d… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 40, 24,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 40, 24,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ conv2d_2[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,160 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 40, 24,    │          0 │ conv2d_2[0][0],   │
│                     │ 64)               │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 40, 24,    │        256 │ multiply[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 20, 12,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d_1  │ (None, 20, 12,    │        640 │ max_pooling2d[0]… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 20, 12,    │      8,320 │ depthwise_conv2d… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 12,    │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 20, 12,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ conv2d_4[0][0]    │
│ (GlobalAveragePool… │                   │            │                 

 Total params: 945,228 (3.61 MB)

 Trainable params: 943,372 (3.60 MB)

 Non-trainable params: 1,856 (7.25 KB)

In [8]:
# Train the model

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val),

                    shuffle=True, callbacks=[lr_scheduler])

Epoch 1/50


I0000 00:00:1730977803.642901      95 service.cc:145] XLA service 0x7cfc68002d10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730977803.642952      95 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  21/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.2719 - loss: 2.5810      

I0000 00:00:1730977811.177206      95 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 30s 8ms/step - accuracy: 0.6139 - loss: 1.5620 - val_accuracy: 0.7337 - val_loss: 1.2077 - learning_rate: 0.0010
Epoch 2/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.7285 - loss: 1.2230 - val_accuracy: 0.7179 - val_loss: 1.2567 - learning_rate: 0.0010
Epoch 3/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.7609 - loss: 1.1298 - val_accuracy: 0.7731 - val_loss: 1.0961 - learning_rate: 0.0010
Epoch 4/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.7927 - loss: 1.0540 - val_accuracy: 0.7902 - val_loss: 1.0595 - learning_rate: 0.0010
Epoch 5/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8172 - loss: 0.9979 - val_accuracy: 0.8067 - val_loss: 1.0190 - learning_rate: 0.0010
Epoch 6/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8384 - loss: 0.9478 - val_accuracy: 0.8313 - val_loss: 0.9635 - learning_rate: 0.0010
Epoch 7/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.8577 - loss:

In [9]:


# Predictions on the test set

y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)

y_test_classes = np.argmax(y_test, axis=1)

350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [10]:
model.save('detect_model_12c.h5')

In [11]:
model.save('model_detect_12c.keras')

In [12]:
new_model1 = tf.keras.models.load_model('detect_model_12c.h5')

In [13]:
X_test[0].shape

(40, 24, 1)

In [14]:
y_test_classes.shape

(11200,)

In [15]:
y_pred.shape

(11200, 12)

In [16]:
p = 0

q=0

for x in range(0,11200):

    if(y_test_classes[x] == np.argmax(y_pred[x])):

        p = p+1

    else:

        q = q+1

In [17]:
p/(p+q)

0.90875

In [18]:
inp_img = np.array(X_test[0])

In [19]:
inp_img = inp_img.reshape(1,40,24,1)

In [20]:
y_pred = new_model1(inp_img)

In [21]:
p=0

q = 0

if(y_test_classes[0] == np.argmax(y_pred)):

    p = p+1

else:

    q = q+1

In [22]:
p/(p+q)

1.0

In [23]:
q

0

In [38]:
new_model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 40, 24, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 40, 24, 32)           320       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 40, 24, 32)           128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 depthwise_conv2d (Depthwis  (None, 40, 24, 32)           320       ['batch_normalization[0][0

In [39]:
y_pred = new_model1(inp_img)

In [42]:
np.argmax(y_pred)

7

In [43]:
y_test_classes[0]

7

In [46]:
ly1 = new_model1.layers[0](inp_img)

In [47]:
ly1.shape

TensorShape([1, 40, 24, 1])

In [50]:
ly2 = new_model1.layers[1](ly1)

ly3 = new_model1.layers[2](ly2)

ly4 = new_model1.layers[3](ly3)

ly5 = new_model1.layers[4](ly4)

ly6 = new_model1.layers[5](ly5)

ly7 = new_model1.layers[6](ly6)

In [51]:
ly7.shape

TensorShape([1, 40, 24, 64])

In [52]:
ly8 = new_model1.layers[7](ly7)

ly9 = new_model1.layers[8](ly8)

ly10 = new_model1.layers[9](ly9)

In [59]:
ly7.shape

TensorShape([1, 40, 24, 64])

In [60]:
ly11 = new_model1.layers[10]([ly7,ly10])

In [62]:
ly11.shape

TensorShape([1, 40, 24, 64])

In [63]:
ly12 = new_model1.layers[11](ly11)

ly12 = new_model1.layers[11](ly12)

ly12 = new_model1.layers[11](ly13)

ly12 = new_model1.layers[11](ly14)

ly12 = new_model1.layers[11](ly15)

ly12 = new_model1.layers[11](ly16)